In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jashwanthreddya/youtube")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\glint\.cache\kagglehub\datasets\jashwanthreddya\youtube\versions\1


In [2]:
import pandas as pd
from  pathlib import Path

In [ ]:
dataset_dir = Path("/Users/hanane/.cache/kagglehub/datasets/jashwanthreddya/youtube/versions/1")
csv_path = next(dataset_dir.glob("*.csv"))

In [3]:
csv_path = Path(r"C:\Users\glint\OneDrive\Documents\Ironhack lessons\Youtube_dataset\DATASET.csv")
print(csv_path)

C:\Users\glint\OneDrive\Documents\Ironhack lessons\Youtube_dataset\DATASET.csv


In [4]:
df = pd.read_csv(csv_path)

In [7]:
#1 csv with the categories:
csv_path1 = Path(r"C:\Users\glint\OneDrive\Documents\Ironhack lessons\Youtube_dataset\video_categories.csv")
#2 csv with the dataset:
csv_path = Path(r"C:\Users\glint\OneDrive\Documents\Ironhack lessons\Youtube_dataset\DATASET.csv")

In [8]:
df_data = pd.read_csv(csv_path)
df_categories = pd.read_csv(csv_path1)

In [17]:
df_new = pd.merge(df_data, df_categories, 
                  on="video_id", 
                  how="left")
df_new.head(1)

,video_id,title,channel_id,channel_title,published_at,view_count,likes,dislikes,comment_count,tags,description,comments,category_id,category_name
0,--0bCF-iK2E,Jadon Sancho Magical Skills & Goals,UC6UL29enLNe4mqwTfAyeNuw,Bundesliga,2021-07-01 10:00:00,1048888,19515,226,1319,football soccer ftbol alemn Bundesliga season ...,Enjoy the best skills and goals from Jadon San...,"Respect to Dortmund fans,must be sad losing hi...",17.0,Sports


In [ ]:
# Cleaning category_id to rid off .0 in the id:
# regex=False means treat ".0" as a plain string, not a regular expression (regex).
# It removes .0 from the end of numbers that look like "17.0" → "17".
df_new["category_id"] = (df_new["category_id"].astype(str).str.replace(".0", "", regex=False))

In [ ]:
#Example to show that for one channel there are two related video categories: ex. Sky Sports Football >> 'Sports' 'Entertainment'.
cat_channel = df_new[df_new["channel_title"] == "Sky Sports Football"]["category_name"].unique()
print(cat_channel)

['Sports' 'Entertainment']


In [32]:
pivot_sum = pd.pivot_table(
    df_new,
    values="view_count",
    index="category_name",
    aggfunc="sum"
)
pivot_pct = (pivot_sum / pivot_sum.values.sum() * 100).round()
pivot_pct_desc = pivot_pct.sort_values(by="view_count", ascending=False)

pivot_pct_desc.head(10)

,view_count
category_name,
Music,47.0
Entertainment,19.0
Gaming,8.0
People & Blogs,7.0
Comedy,5.0
Sports,5.0
Film & Animation,2.0
Howto & Style,2.0
Science & Technology,2.0


In [51]:
#2 bis/among these categories :  
# The ratio of dislikes to total reactions (likes + dislikes) is the same across categories.
pivot_sum = pd.pivot_table(
    df_new,
    values=["likes", "dislikes"],
    index="category_name",
    aggfunc="sum"
)

pivot_sum["total_reactions"] = (pivot_sum["likes"] + pivot_sum["dislikes"])

pivot_sum["ratio_%"] = (pivot_sum["dislikes"] / pivot_sum["total_reactions"] * 100).round(2)
total_dislikes = pivot_sum["dislikes"].sum()

pivot_sum["ratio_dislikes"] = (
    pivot_sum["dislikes"] / total_dislikes * 100
).round(2)

pivot_sum[['total_reactions', 'ratio_%', 'ratio_dislikes']].sort_values("ratio_%", ascending=False)

,total_reactions,ratio_%,ratio_dislikes
category_name,,,
News & Politics,28089169,9.35,1.48
Howto & Style,122269470,3.80,2.61
Comedy,451512485,3.60,9.13
People & Blogs,624184981,3.28,11.51
Nonprofits & Activism,5583870,3.19,0.10
Sports,241763600,3.01,4.09
Entertainment,1451685650,2.87,23.42
Music,2140064795,2.76,33.15
Autos & Vehicles,48296943,2.63,0.71
